# Toy problem for TLS solution
We want to solve here the identification problem with TLS in a very simple case: a network with 2 nodes, without any structural contraint.

In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp
import mlflow

In [ ]:
import sys
sys.path.insert(1, '..')

from src.models.error_in_variable import TotalLeastSquares, SparseTotalLeastSquare
from src.models.regression import ComplexLasso
from src.identification.error_metrics import fro_error, error_metrics

In [ ]:
np.set_printoptions(precision=2)
mlflow.set_experiment('Toy problem with Cartesian noise (4 nodes)')

In [ ]:
samples = 300
v_noise_sd = 0.01
i_noise_sd = 0.01
mlflow_params = {'samples': samples, 'v_noise_sd': v_noise_sd, 'i_noise_sd': i_noise_sd}

In [ ]:
np.random.seed(11)
y_bus = np.array([
    [1+1j, 0, 0, 0],
    [0, 2+1j, 0, 0],
    [2+1j, 0, 1+1j, 0],
    [0, 0, 0, 2+1j],
])
nodes = y_bus.shape[0]
real_voltages = np.random.normal(1, 0.1, (samples, nodes)) + 1j*np.random.normal(1, 0.1, (samples, nodes))
real_currents = real_voltages @ y_bus

voltages = real_voltages.copy() + np.random.normal(0, v_noise_sd, (samples, nodes)) + 1j*np.random.normal(0, v_noise_sd, (samples, nodes))
currents = real_currents.copy() + np.random.normal(0, i_noise_sd, (samples, nodes)) + 1j*np.random.normal(0, i_noise_sd, (samples, nodes))

voltage_error = voltages - real_voltages
current_error = currents - real_currents

# Standard TLS

In [ ]:
mlflow.start_run(run_name='TLS')
eiv = TotalLeastSquares()
eiv.fit(voltages, currents)
y_eiv = eiv.fitted_admittance_matrix
y_eiv

In [ ]:
tls_metrics = error_metrics(y_bus, y_eiv)
mlflow.log_params(mlflow_params)
mlflow.log_metrics(tls_metrics.__dict__)
mlflow.end_run()

# Standard LASSO

In [ ]:
mlflow.start_run(run_name='Lasso')
lasso = ComplexLasso(y_bus, verbose=False, lambdas=np.logspace(-4, 0, 20))
lasso.fit(voltages, currents)
y_lasso = lasso.fitted_admittance_matrix
y_lasso

In [ ]:
lasso_metrics = error_metrics(y_bus, y_lasso)
mlflow.log_param('lambda', lasso.best_trial.hyperparameters['lambda'])
mlflow.log_params(mlflow_params)
mlflow.log_metrics(lasso_metrics.__dict__)
mlflow.end_run()

In [ ]:
lasso.best_trial

# L1-regularized TLS

In [ ]:
mlflow.start_run(run_name='S-TLS')
tls_lambda = lasso.best_trial.hyperparameters['lambda']
max_iterations = 50
abs_tol = 10e-12
rel_tol = 10e-12
solver = cp.GUROBI
use_l1_penalty = False
sparse_eiv = SparseTotalLeastSquare(lambda_value=tls_lambda, abs_tol=abs_tol, rel_tol=rel_tol, solver=solver, max_iterations=max_iterations, use_l1_penalty=use_l1_penalty)
sparse_eiv.fit(voltages, currents)
y_sparse_eiv = sparse_eiv.fitted_admittance_matrix

In [ ]:
y_sparse_eiv

In [ ]:
stls_metrics = error_metrics(y_bus, y_sparse_eiv)
stls_metrics

In [ ]:
y_errors = pd.Series([fro_error(y_bus, i.fitted_parameters) for i in sparse_eiv.iterations])
y_errors.plot(title='Fro error on Y');

In [ ]:
targets = pd.Series([i.target_function for i in sparse_eiv.iterations])
targets.plot(title='Target function');

In [ ]:
mlflow.log_param('lambda', tls_lambda)
mlflow.log_param('max_iterations', max_iterations)
mlflow.log_param('abs_tol', abs_tol)
mlflow.log_param('rel_tol', rel_tol)
mlflow.log_param('solver', solver)
mlflow.log_param('use_l1_penalty', use_l1_penalty)
mlflow.log_params(mlflow_params)
mlflow.log_metrics(stls_metrics.__dict__)
for i in range(len(y_errors)):
    mlflow.log_metric('fro_error_evo', value=y_errors[i], step=i)
    mlflow.log_metric('opt_cost_evo', value=targets[i], step=i)
mlflow.end_run()

In [ ]:
np.abs(y_bus - y_sparse_eiv)

In [ ]:
np.abs(y_bus - y_eiv)

In [ ]:
np.abs(y_bus - y_sparse_eiv) > np.abs(y_bus - y_eiv)